In [1]:
import pandas as pd
import numpy as np
import itertools

In [2]:
bakery1000 = pd.read_csv('../apriori/1000/1000-out1.csv',sep = '\n', header = None)
bakery1000Mtemp = bakery1000.iloc[:,0].values
bakery1000M = [list(map(int, x.split(','))) for x in bakery1000Mtemp]
for x in bakery1000M:
    del x[0]

In [3]:
goods = pd.read_csv('../apriori/goods.csv')
goods['Flavor'] = goods['Flavor'] + "-" + goods['Food']
goodsDict = dict(zip(goods['Id'], goods['Flavor']))

In [4]:
exampleData = pd.read_csv('../example/out1.csv', sep = '\n', header = None)
exampleDataTemp = exampleData.iloc[:,0].values
exampleDataM = [list(map(int, x.split(','))) for x in exampleDataTemp]
for x in exampleDataM:
    del x[0]

In [5]:
def support(item, baskets):
    top = sum([1 if item in basket else 0 for basket in baskets])
    bottom = len(baskets)
    return top*1.0/bottom

In [6]:
def getPairs(F, k):
    outList = []
    for first in F:
        for second in F:
            if len(first) == len(second) == k:
                outList.append([first, second, tuple(set(first) | set(second))])
    return outList 

In [7]:
def getSubsets(tmpTuple):
    size = len(tmpTuple) - 1
    return set(itertools.combinations(list(tmpTuple), size))

In [8]:
def candidate(F,k):
    C = set()
    for pair in getPairs(F,k):
        if len(pair[2]) == len(pair[0]) + 1:
            c = pair[2]
            flag = True
            for s in getSubsets(c):
                if s not in F:
                    flag = False
            if flag == True:
                if c not in C:
                    C.add(tuple(c))
    return C

In [9]:
def apriori(T, minSup):
    FullSupport = []
    I =  list(set(x for l in T for x in l))
    Fk = []; Fk.append(None);
    firstIter = set(tuple([x]) for x in filter(lambda a: a != -1, [x if support(x, T) >= minSup else -1 for x in I]))
    Fk.append(firstIter)
    k = 2
    while len(Fk[k-1]) != 0:
        Ck = candidate(Fk[k-1], k-1)
        count = {}
        for c in Ck:
            count[c] = 0
        for t in T:
            for c in Ck:
                if set(c) <= set(tuple(t)):
                    count[c] += 1
        tmpFk = []
        for c in Ck:
            if count[c]*1.0/len(T) >= minSup:
                tmpFk.append(c)
        Fk.append(set(tmpFk))
        newSet = set(x for x in Fk[k-1])
        for newTerm in Fk[k]:
            for oldTerm in Fk[k-1]:
                if set(oldTerm).issubset(newTerm) == True:
                    if oldTerm in newSet:
                        newSet.remove(oldTerm)
                        #print("After: ", newSet)
        Fk[k-1] = set(tuple(sorted(x)) for x in newSet)
        k += 1
    Fk = Fk[1:-1]
    return Fk
                    
            
        
    
    

In [10]:
def getCorrect():
    t = np.linspace(.05, .01, 20)
    for minS in t:
        ap = addNamesFreq(apriori(bakery1000M, minS),goodsDict)
        print("---- Apriori: ", minS, " ----")
        print("NumRules: ", len(genRules(bakery1000M, ap, .85)))
        for i, val in enumerate(ap):
            print(i+1, ": ", len(val))
            

In [11]:
def genRules(data, freqItemSet, minConf):
    allRules = []
    freqItemSet = freqItemSet[1:]
    freqItemSet = [item for sublist in freqItemSet for item in sublist]
    for itemSet in freqItemSet:
        for item in itemSet:
            noItem=set(itemSet)-(set([item]))
            item = set([item])
            conf = confidence(data,noItem,item)
            sup = supportOut(data,noItem,item)
            confItem = [noItem, item, conf, sup]
            if conf >= minConf:
                allRules.append(confItem)
                
            
    return allRules
                    

In [12]:
def confidence(T,set1,set2): #Dataset,leftSideList,rightSideList
    top = sum([1 if (set1|set2) <= set(basket) else 0 for basket in T])
    bottom = sum([1 if (set1) <= set(basket) else 0 for basket in T])
    if(bottom == 0 | top == 0):
        return 0
    return top*1.0/bottom

In [13]:
def supportOut(T, set1,set2):
    return sum([1 if (set1|set2) <= set(basket) else 0 for basket in T])/len(T)

In [14]:
tmpOut = apriori(bakery1000M, .01)

In [15]:
def getSupportFreq(T, output):
    allFreq = []
    for groupSize in output:
        for term in groupSize:
            termSup = supportOut(T, set(term), set([]))
            termOut = [term,termSup]
            allFreq.append(termOut)
    return allFreq
            
    

In [16]:
def addNamesFreq(output, goods):
    groupList=[]
    for i, basket in enumerate(output):
        basketList = []
        for value in list(basket[0]):
            basketList.append(goods[value])
        groupList.append([basketList, basket[1]])
    return groupList

In [17]:
def addNames(output, goods):
    ruleList = []
    for i, rule in enumerate(output):
        termList = []
        for j, terms in enumerate(rule[:-2]):
            setList = []
            for k, objectId in enumerate(list(terms)):
                setList.append(goods[objectId])
            termList.append(setList)
        termList.append(rule[2])
        termList.append(rule[3])
        ruleList.append(termList)
    return ruleList  

In [18]:
def printCorrectly(output):
    for i, rule in enumerate(output):
        print("Rule: ", i, '   ', ','.join(rule[0]), " --> ", 
                rule[1][0]) 
        print("              Support: ", str(rule[3]))
        print("              Confidence: ", str(rule[2]))
        

In [19]:
def findRules(T, minSup, minConf, goodsDictTmp, bakery = False):
    aprioriOutput = apriori(T, minSup)
    aprioriNames = None
    if(bakery == True):
        aprioriNames = addNamesFreq(getSupportFreq(T, aprioriOutput), goodsDictTmp)
        for i, itemSet in enumerate(aprioriNames):
            print('Set # ', i+1, " ", itemSet)
    genRulesOutput = genRules(T, aprioriOutput, minConf)
    if(bakery == True):
        genRulesOutput = addNames(genRulesOutput, goodsDictTmp)
    print("**********************************************")
    printCorrectly(genRulesOutput)
    

In [20]:
findRules(bakery1000M, .018, .9, goodsDict, bakery=True)

Set #  1   [["'Vanilla'-'Meringue'"], 0.047]
Set #  2   [["'Ganache'-'Cookie'"], 0.044]
Set #  3   [["'Chocolate'-'Croissant'"], 0.042]
Set #  4   [["'Vanilla'-'Eclair'"], 0.037]
Set #  5   [["'Almond'-'Croissant'"], 0.049]
Set #  6   [["'Almond'-'Tart'"], 0.041]
Set #  7   [["'Almond'-'Bear Claw'"], 0.026]
Set #  8   [["'Chocolate'-'Eclair'"], 0.034]
Set #  9   [["'Blueberry'-'Danish'"], 0.055]
Set #  10   [["'Apricot'-'Tart'"], 0.056]
Set #  11   [["'Chocolate'-'Meringue'"], 0.038]
Set #  12   [["'Pecan'-'Tart'"], 0.04]
Set #  13   [["'Strawberry'-'Cake'", "'Napoleon'-'Cake'"], 0.049]
Set #  14   [["'Opera'-'Cake'", "'Apricot'-'Danish'"], 0.039]
Set #  15   [["'Cherry'-'Tart'", "'Apricot'-'Danish'"], 0.046]
Set #  16   [["'Berry'-'Tart'", "'Bottled'-'Water'"], 0.034]
Set #  17   [["'Opera'-'Cake'", "'Cherry'-'Tart'"], 0.041]
Set #  18   [["'Lemon'-'Cake'", "'Lemon'-'Tart'"], 0.04]
Set #  19   [["'Cheese'-'Croissant'", "'Orange'-'Juice'"], 0.038]
Set #  20   [["'Truffle'-'Cake'", "'Go

In [21]:
bakery5000 = pd.read_csv('../apriori/5000/5000-out1.csv',sep = '\n', header = None)
bakery5000Mtemp = bakery5000.iloc[:,0].values
bakery5000M = [list(map(int, x.split(','))) for x in bakery5000Mtemp]
for x in bakery5000M:
    del x[0]

In [34]:
def findMinSupport(dataSet,goodsDict):
    t = np.linspace(.05, .01, 5)
    for minSup in t:
        print("******* ",minSup," *******")
        findRules(dataSet, minSup, .8, goodsDict, bakery=True)
    

In [25]:
findRules(bakery5000M, .02, .9, goodsDict, bakery=True)

Set #  1   [["'Vanilla'-'Meringue'"], 0.0398]
Set #  2   [["'Ganache'-'Cookie'"], 0.0388]
Set #  3   [["'Chocolate'-'Croissant'"], 0.0432]
Set #  4   [["'Vanilla'-'Eclair'"], 0.046]
Set #  5   [["'Almond'-'Croissant'"], 0.0456]
Set #  6   [["'Almond'-'Tart'"], 0.0386]
Set #  7   [["'Almond'-'Bear Claw'"], 0.0428]
Set #  8   [["'Chocolate'-'Eclair'"], 0.0382]
Set #  9   [["'Blueberry'-'Danish'"], 0.04]
Set #  10   [["'Apricot'-'Tart'"], 0.0422]
Set #  11   [["'Chocolate'-'Meringue'"], 0.0452]
Set #  12   [["'Pecan'-'Tart'"], 0.0444]
Set #  13   [["'Strawberry'-'Cake'", "'Napoleon'-'Cake'"], 0.0422]
Set #  14   [["'Opera'-'Cake'", "'Apricot'-'Danish'"], 0.0432]
Set #  15   [["'Cherry'-'Tart'", "'Apricot'-'Danish'"], 0.0512]
Set #  16   [["'Berry'-'Tart'", "'Bottled'-'Water'"], 0.0366]
Set #  17   [["'Opera'-'Cake'", "'Cherry'-'Tart'"], 0.0436]
Set #  18   [["'Lemon'-'Cake'", "'Lemon'-'Tart'"], 0.0336]
Set #  19   [["'Cheese'-'Croissant'", "'Orange'-'Juice'"], 0.043]
Set #  20   [["'Truff

In [26]:
bakery20000 = pd.read_csv('../apriori/20000/20000-out1.csv',sep = '\n', header = None)
bakery20000Mtemp = bakery20000.iloc[:,0].values
bakery20000M = [list(map(int, x.split(','))) for x in bakery20000Mtemp]
for x in bakery20000M:
    del x[0]

In [32]:
findRules(bakery20000M, .02, .9, goodsDict, bakery=True)

Set #  1   [["'Vanilla'-'Meringue'"], 0.0424]
Set #  2   [["'Ganache'-'Cookie'"], 0.0433]
Set #  3   [["'Chocolate'-'Croissant'"], 0.0446]
Set #  4   [["'Vanilla'-'Eclair'"], 0.0427]
Set #  5   [["'Almond'-'Croissant'"], 0.04205]
Set #  6   [["'Almond'-'Tart'"], 0.04055]
Set #  7   [["'Almond'-'Bear Claw'"], 0.04425]
Set #  8   [["'Chocolate'-'Eclair'"], 0.0426]
Set #  9   [["'Blueberry'-'Danish'"], 0.04115]
Set #  10   [["'Apricot'-'Tart'"], 0.04275]
Set #  11   [["'Chocolate'-'Meringue'"], 0.0445]
Set #  12   [["'Pecan'-'Tart'"], 0.04155]
Set #  13   [["'Strawberry'-'Cake'", "'Napoleon'-'Cake'"], 0.04455]
Set #  14   [["'Opera'-'Cake'", "'Apricot'-'Danish'"], 0.04335]
Set #  15   [["'Cherry'-'Tart'", "'Apricot'-'Danish'"], 0.05255]
Set #  16   [["'Berry'-'Tart'", "'Bottled'-'Water'"], 0.0357]
Set #  17   [["'Opera'-'Cake'", "'Cherry'-'Tart'"], 0.04365]
Set #  18   [["'Lemon'-'Cake'", "'Lemon'-'Tart'"], 0.037]
Set #  19   [["'Cheese'-'Croissant'", "'Orange'-'Juice'"], 0.0439]
Set #  2

In [33]:
bakery75000 = pd.read_csv('../apriori/75000/75000-out1.csv',sep = '\n', header = None)
bakery75000Mtemp = bakery75000.iloc[:,0].values
bakery75000M = [list(map(int, x.split(','))) for x in bakery75000Mtemp]
for x in bakery75000M:
    del x[0]

In [36]:
findMinSupport(bakery75000M,goodsDict)

*******  0.05  *******
Set #  1   [["'Apricot'-'Croissant'"], 0.08398666666666667]
Set #  2   [["'Raspberry'-'Cookie'"], 0.06764]
Set #  3   [["'Napoleon'-'Cake'"], 0.08274666666666666]
Set #  4   [["'Apple'-'Danish'"], 0.06769333333333333]
Set #  5   [["'Apple'-'Pie'"], 0.07712]
Set #  6   [["'Single'-'Espresso'"], 0.06797333333333333]
Set #  7   [["'Lemon'-'Cookie'"], 0.06801333333333333]
Set #  8   [["'Blackberry'-'Tart'"], 0.07586666666666667]
Set #  9   [["'Lemon'-'Cake'"], 0.08361333333333333]
Set #  10   [["'Opera'-'Cake'"], 0.08209333333333334]
Set #  11   [["'Raspberry'-'Lemonade'"], 0.06774666666666666]
Set #  12   [["'Truffle'-'Cake'"], 0.08224]
Set #  13   [["'Blueberry'-'Tart'"], 0.08294666666666667]
Set #  14   [["'Green'-'Tea'"], 0.06246666666666666]
Set #  15   [["'Coffee'-'Eclair'"], 0.10924]
Set #  16   [["'Hot'-'Coffee'"], 0.10266666666666667]
Set #  17   [["'Vanilla'-'Frappuccino'"], 0.07746666666666667]
Set #  18   [["'Gongolais'-'Cookie'"], 0.09044]
Set #  19   [[

Set #  1   [["'Vanilla'-'Meringue'"], 0.04238666666666667]
Set #  2   [["'Ganache'-'Cookie'"], 0.04324]
Set #  3   [["'Chocolate'-'Croissant'"], 0.04324]
Set #  4   [["'Vanilla'-'Eclair'"], 0.04252]
Set #  5   [["'Almond'-'Croissant'"], 0.04273333333333333]
Set #  6   [["'Almond'-'Tart'"], 0.04204]
Set #  7   [["'Almond'-'Bear Claw'"], 0.04244]
Set #  8   [["'Chocolate'-'Eclair'"], 0.04237333333333333]
Set #  9   [["'Blueberry'-'Danish'"], 0.04409333333333333]
Set #  10   [["'Apricot'-'Tart'"], 0.04236]
Set #  11   [["'Chocolate'-'Meringue'"], 0.041933333333333336]
Set #  12   [["'Pecan'-'Tart'"], 0.04337333333333333]
Set #  13   [["'Strawberry'-'Cake'", "'Napoleon'-'Cake'"], 0.043146666666666667]
Set #  14   [["'Opera'-'Cake'", "'Apricot'-'Danish'"], 0.043026666666666664]
Set #  15   [["'Cherry'-'Tart'", "'Apricot'-'Danish'"], 0.05309333333333333]
Set #  16   [["'Berry'-'Tart'", "'Bottled'-'Water'"], 0.0378]
Set #  17   [["'Opera'-'Cake'", "'Cherry'-'Tart'"], 0.04337333333333333]
Set 

Set #  1   [["'Vanilla'-'Meringue'"], 0.04238666666666667]
Set #  2   [["'Ganache'-'Cookie'"], 0.04324]
Set #  3   [["'Chocolate'-'Croissant'"], 0.04324]
Set #  4   [["'Vanilla'-'Eclair'"], 0.04252]
Set #  5   [["'Almond'-'Croissant'"], 0.04273333333333333]
Set #  6   [["'Almond'-'Tart'"], 0.04204]
Set #  7   [["'Almond'-'Bear Claw'"], 0.04244]
Set #  8   [["'Chocolate'-'Eclair'"], 0.04237333333333333]
Set #  9   [["'Blueberry'-'Danish'"], 0.04409333333333333]
Set #  10   [["'Apricot'-'Tart'"], 0.04236]
Set #  11   [["'Chocolate'-'Meringue'"], 0.041933333333333336]
Set #  12   [["'Pecan'-'Tart'"], 0.04337333333333333]
Set #  13   [["'Strawberry'-'Cake'", "'Napoleon'-'Cake'"], 0.043146666666666667]
Set #  14   [["'Opera'-'Cake'", "'Apricot'-'Danish'"], 0.043026666666666664]
Set #  15   [["'Cherry'-'Tart'", "'Apricot'-'Danish'"], 0.05309333333333333]
Set #  16   [["'Berry'-'Tart'", "'Bottled'-'Water'"], 0.0378]
Set #  17   [["'Opera'-'Cake'", "'Cherry'-'Tart'"], 0.04337333333333333]
Set 

In [37]:
findRules(bakery75000M, .02, .9, goodsDict, bakery=True)

Set #  1   [["'Vanilla'-'Meringue'"], 0.04238666666666667]
Set #  2   [["'Ganache'-'Cookie'"], 0.04324]
Set #  3   [["'Chocolate'-'Croissant'"], 0.04324]
Set #  4   [["'Vanilla'-'Eclair'"], 0.04252]
Set #  5   [["'Almond'-'Croissant'"], 0.04273333333333333]
Set #  6   [["'Almond'-'Tart'"], 0.04204]
Set #  7   [["'Almond'-'Bear Claw'"], 0.04244]
Set #  8   [["'Chocolate'-'Eclair'"], 0.04237333333333333]
Set #  9   [["'Blueberry'-'Danish'"], 0.04409333333333333]
Set #  10   [["'Apricot'-'Tart'"], 0.04236]
Set #  11   [["'Chocolate'-'Meringue'"], 0.041933333333333336]
Set #  12   [["'Pecan'-'Tart'"], 0.04337333333333333]
Set #  13   [["'Strawberry'-'Cake'", "'Napoleon'-'Cake'"], 0.043146666666666667]
Set #  14   [["'Opera'-'Cake'", "'Apricot'-'Danish'"], 0.043026666666666664]
Set #  15   [["'Cherry'-'Tart'", "'Apricot'-'Danish'"], 0.05309333333333333]
Set #  16   [["'Berry'-'Tart'", "'Bottled'-'Water'"], 0.0378]
Set #  17   [["'Opera'-'Cake'", "'Cherry'-'Tart'"], 0.04337333333333333]
Set 